<a href="https://colab.research.google.com/github/Yiming-xie-2021/aiperson_utility/blob/main/ms_swift_%E9%AD%94%E6%90%AD%2Bcolab_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install 'ms-swift[all]' -U
!pip install bitsandbytes

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 kB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.7 MB/s eta 0:00:00
^C


In [ ]:
#打开网址 https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64 下载后上传
!pwd
# 1. 强制创建目标目录（如果不存在的话）
!mkdir -p /root/.cache/huggingface/gradio/frpc

# 2. 将文件复制过去并改名为带版本号的名称
!cp /mnt/workspace/frpc_linux_amd64 /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3

# 3. 赋予可执行权限
!chmod +x /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3

# 4. 验证是否成功（如果显示 -rwxr-xr-x 且文件名正确就没问题了）
!ls -l /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3

/mnt/workspace
-rwxr-xr-x 1 root root 2097152 12月 21 12:02 /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3


In [ ]:
!swift web-ui --lang zh --share

run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/web_ui.py --lang zh --share`
[INFO:swift] Successfully registered `/usr/local/lib/python3.11/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] args: WebUIArguments(server_name='0.0.0.0', server_port=7860, share=True, lang='zh')
[INFO:swift] Start time of running main: 2025-12-19 14:24:43.590645
[INFO:swift] swift.__version__: 3.11.1
/usr/local/lib/python3.11/site-packages/swift/ui/llm_infer/llm_infer.py:145: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(elem_id='chatbot', elem_classes='control-height')
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://f76a7a4f6392f762af.gradio.live


In [ ]:
import json
from modelscope import AutoTokenizer

# 你可以用 Qwen2 或 Qwen3 的 tokenizer，统计长度都没问题
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B",
    trust_remote_code=True
)

def count_tokens(messages):
    text = ""
    for turn in messages:
        role = turn.get("role", "")
        content = turn.get("content", "")
        text += f"{role}: {content}\n"
    return len(tokenizer.encode(text))

input_path = "/mnt/workspace/sharegpt_output.jsonl"
output_path = "/mnt/workspace/sharegpt_filtered.jsonl"
max_tokens = 2048

idx = 0
kept = 0
removed = 0

with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
    for line in fin:
        idx += 1
        data = json.loads(line)
        messages = data.get("messages", [])
        token_count = count_tokens(messages)

        print(f"[{idx}] tokens={token_count}")

        if token_count <= max_tokens:
            fout.write(json.dumps(data, ensure_ascii=False) + "\n")
            kept += 1
        else:
            removed += 1

print("\n=== Summary ===")
print(f"Total: {idx}")
print(f"Kept: {kept}")
print(f"Removed: {removed}")


[1] tokens=2479
[2] tokens=2564
[3] tokens=2474
[4] tokens=2518
[5] tokens=1169
[6] tokens=1242
[7] tokens=1352
[8] tokens=1173
[9] tokens=1324
[10] tokens=1263
[11] tokens=1351
[12] tokens=1345
[13] tokens=1237
[14] tokens=1374
[15] tokens=1298
[16] tokens=1264
[17] tokens=1262
[18] tokens=3298
[19] tokens=1250
[20] tokens=3788
[21] tokens=3711
[22] tokens=3090
[23] tokens=2983
[24] tokens=3623
[25] tokens=3699
[26] tokens=3761
[27] tokens=3089
[28] tokens=3036
[29] tokens=3249
[30] tokens=2603
[31] tokens=2758
[32] tokens=3032
[33] tokens=3026
[34] tokens=3946
[35] tokens=3829
[36] tokens=3915
[37] tokens=4011
[38] tokens=2441
[39] tokens=2818
[40] tokens=2260
[41] tokens=1350
[42] tokens=2743
[43] tokens=1293
[44] tokens=1340
[45] tokens=1192
[46] tokens=1430
[47] tokens=1288
[48] tokens=1290
[49] tokens=1276
[50] tokens=2646
[51] tokens=1177
[52] tokens=1338
[53] tokens=2856
[54] tokens=1274
[55] tokens=1441
[56] tokens=1172
[57] tokens=944
[58] tokens=1102
[59] tokens=1241
[60] to

In [ ]:
!CUDA_VISIBLE_DEVICES=0 \
 swift sft \
    --model Qwen/Qwen3-8B \
    --train_type lora \
    --dataset /mnt/workspace/sharegpt_filtered.jsonl \
    --torch_dtype bfloat16 \
    --bnb_4bit_compute_dtype bfloat16 \
    --bnb_4bit_quant_type nf4 \
    --bnb_4bit_use_double_quant true \
    --quant_method bnb \
    --quant_bits 4 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32 \
    --target_modules all-linear \
    --gradient_accumulation_steps 16 \
    --eval_steps 50 \
    --save_steps 50 \
    --save_total_limit 2 \
    --logging_steps 5 \
    --max_length 2048 \
    --output_dir output \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 4 \
    --model_author swift \
    --model_name ai-person

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:56: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/sft.py --model Qwen/Qwen3-8B --train_type lora --dataset /mnt/workspace/sharegpt_filtered.jsonl --torch_dtype bfloat16 --bnb_4bit_compute_dtype bfloat16 --bnb_4bit_quant_type nf4 --bnb_4bit_use_double_quant true --quant_method bnb --quant_bits 4 --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --learning_rate 1e-4 --lora_rank 8 --lora_alpha 32 --target_modules all-linear --gradient_accumulation_steps 16 --eval_steps 50 --save_steps 50 --save_total_limit 2 --logging_steps 5 --max_length 2048 --output_dir output --warmup_ratio 0.05 --dataloader_num_workers 4 --mo

In [ ]:
# --adapters output/v0-20251221-122053/checkpoint-12 \

def chat(prompt):
    cmd = f"""
    CUDA_VISIBLE_DEVICES=0 swift infer \
        --model Qwen/Qwen3-8B \
        --prompt "{prompt}" \
        --temperature 0 \
        --max_new_tokens 2048
    """
    !{cmd}

chat("你是谁？")



In [ ]:
!cd /mnt/workspace && zip -r output.zip output #压缩lora

  adding: output/ (stored 0%)
  adding: output/v0-20251221-122053/ (stored 0%)
  adding: output/v0-20251221-122053/runs/ (stored 0%)
  adding: output/v0-20251221-122053/runs/events.out.tfevents.1766290997.eais-bjh1ari71pyj7xswquht-0.248.0 (deflated 65%)
  adding: output/v0-20251221-122053/images/ (stored 0%)
  adding: output/v0-20251221-122053/images/train_epoch.png (deflated 10%)
  adding: output/v0-20251221-122053/images/train_token_acc.png (deflated 11%)
  adding: output/v0-20251221-122053/images/train_train_steps_per_second.png (deflated 26%)
  adding: output/v0-20251221-122053/images/train_train_loss.png (deflated 25%)
  adding: output/v0-20251221-122053/images/train_loss.png (deflated 11%)
  adding: output/v0-20251221-122053/images/train_train_samples_per_second.png (deflated 24%)
  adding: output/v0-20251221-122053/images/train_train_runtime.png (deflated 23%)
  adding: output/v0-20251221-122053/images/train_total_flos.png (deflated 25%)
  adding: output/v0-20251221-122053/image

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: output/v0-20251221-122053/checkpoint-12/adapter_model.safetensors (deflated 8%)
  adding: output/v0-20251221-122053/checkpoint-12/args.json (deflated 73%)
  adding: output/v0-20251221-122053/args.json (deflated 73%)
  adding: output/v0-20251221-122053/logging.jsonl (deflated 65%)


In [6]:
!cd /content/drive/MyDrive/Colab_Notebooks && unzip -o output.zip


Archive:  output.zip
   creating: output/
   creating: output/v0-20251221-122053/
   creating: output/v0-20251221-122053/runs/
  inflating: output/v0-20251221-122053/runs/events.out.tfevents.1766290997.eais-bjh1ari71pyj7xswquht-0.248.0  
   creating: output/v0-20251221-122053/images/
  inflating: output/v0-20251221-122053/images/train_epoch.png  
  inflating: output/v0-20251221-122053/images/train_token_acc.png  
  inflating: output/v0-20251221-122053/images/train_train_steps_per_second.png  
  inflating: output/v0-20251221-122053/images/train_train_loss.png  
  inflating: output/v0-20251221-122053/images/train_loss.png  
  inflating: output/v0-20251221-122053/images/train_train_samples_per_second.png  
  inflating: output/v0-20251221-122053/images/train_train_runtime.png  
  inflating: output/v0-20251221-122053/images/train_total_flos.png  
  inflating: output/v0-20251221-122053/images/train_learning_rate.png  
  inflating: output/v0-20251221-122053/images/train_grad_norm.png  
   cre

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = "Qwen/Qwen3-8B"
lora = "/content/drive/MyDrive/Colab_Notebooks/output/v0-20251221-122053/checkpoint-12"

tokenizer = AutoTokenizer.from_pretrained(base, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    base,
    trust_remote_code=True,
    device_map="auto",
    load_in_4bit=True,
)

model = PeftModel.from_pretrained(model, lora)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [11]:
prompt = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": "你是一名地产地图直播领域的专家，长期服务于用户购房决策，擅长根据给定的【背景资料】回答用户提出的问题：\n- 从用户的问题和背景资料识别购房者真实意图与隐性焦虑\n- 提炼这类购房人群的共性问题\n- 根据背景资料用专业但不说教的方式给出能改变认知层级的答案\n你的目标不是简单“答题”，而是帮助一类购房用户意识到认知盲区、规避决策风险，并引导其进一步咨询"},
        {"role": "user", "content": "虹口对曲阳板块的更新提升有哪些具体的规划和措施\n\n【背景资料】虹口下滑最严重的地方之一是曲阳。曲阳和大柏树曾是虹口的重点发展区域，曾是产业中心和中产居住地，吸引了众多教师、公务员和产业人群入住，形成了全区最好的居住环境。然而，近年来，虹口全力打造北外滩，导致中北部一些传统居住板块下滑，曲阳便是其中之一。\n\n曲阳的现状不容乐观，面临多方面问题，如二手房产品老化、人口结构失衡、产业流失等。曲阳板块产品不佳，以老破小、老大楼为主，缺乏好房子，是板块竞争力下滑的首要推手。曲阳房龄最高的老破小是50年代建成的玉田新村，其它老破小多在80年代建成。2000年以后建设的“次新”小区非常少，只有北宸佳苑、曲阳豪庭、复城国际、通德苑等少数几个小区。\n\n这些小区规模不大，购买客群多为周边从老公房、老大楼向商品房置换的地缘人群。其中，北宸佳苑、曲阳豪庭是对口虹口四中心小学+虹口实验学校初中的双XQ次新房，地位在板块最高，挂牌单价也处在第一梯队。\n\n曲阳板块的其它产品主要是老破小和老大楼。在众多老破小中，银联花园、东四小区、东五小区、邮电二村、双东小区、建工天峰公寓也是对口虹口四中心小学+虹口实验学校初中的双XQ房。这些小区处在购买价值的第二梯队，成交价在5.4-6万元/m²。\n\n曲阳板块的老破小太多，大部分老公房XQ属性一般、只有自住价值，成交价在4.5-5万元/m²。对于自住的曲阳老破小来说，舒适度比其它片区的老破小更低。但凡是老破小，都具有设施老化、物业管理低下、车位配比少等的通病。\n\n曲阳的老破小劣势更明显，很多小区更小、车位配比更少，市场竞争力下滑更严重。虹口本身地域比较小，开发的小区也普遍偏小。比如松花小区仅295户，小区很小，车位配比还低，只有1：0.07；祥东小区车位配比1：0.06更低，根本没地方停车。\n\n缺点太多后，曲阳的老破小跌幅较大，超过40%。曲阳的老破小在曲阳买个老破小基本上要5万元/m²左右。但同样是中环内，花同样的价格可以在宝山大华买到2000年后还是带电梯的商品房，更加削弱了曲阳板块的竞争力。\n\n比如大华公园世家2001年的电梯商品房，才5.3万元/m²。在区域发展逻辑改变、拆迁预算束缚、军产限制和房龄没到的情况下，曲阳老破小非常缺乏拆迁的预期。\n\n虹口GDP长期在市区垫底，有限的城市更新预算都集中在了北外滩和四川北路区域。虹口南部是城市更新的重点，中北部的曲阳相对式微，老破小缺乏拆迁预期。\n\n板块还有部分军产房不好动，而且80年代的老房子还没到拆迁的周期，更加拖累了曲阳的更新。徐汇有钱、有魅力，拆迁力度走在了全市前列。现在买老破小搏动迁的大多出现在徐汇，东安一二村、江南新村都是幸运的老破小。\n\n区域人口老龄化严重，本地有钱人在流失，没有外来年轻人补充，居住阶层固化，导致曲阳的房子流通性、商业和教育质量都在下滑。虹口是上海老龄化程度最高的区，45.2%的户籍人口在60岁以上，远超上海市37.6%的平均水平。\n\n虹口的人口结构呈现“倒金字塔”形态，劳动年龄人口持续减少，老年抚养比高达49.3%（近2名劳动力需赡养1名65岁以上老人）。同时，虹口近十年人口流失超15万，外来人口吸纳能力全市最低，加剧了老龄化压力。\n\n一部分年轻人和有钱人改善去往了外区，一部分是因为北外滩等区域的动迁彻底离开虹口。本地人在减少，却没有外来年轻人形成有效补充，让本地的购买力在减弱，降低了二手房的流通性。\n\n曲阳板块只是虹口的一个缩影，正是因为缺乏好房子，大家都把房子买到了外区，购买力在流失。不管是稀缺的次新房还是海量的老破小，换手率普遍低迷，市场高度“内生化”“内卷化”。\n\n曲阳板块面临重度的老龄化，老年人购房欲望低、消费欲望也低，进而引发商业衰退。一般浦西老破小板块的烟火气非常浓，但在曲阳不是，沿街的店铺比较少。\n\n板块内没有好的商业，百联曲阳购物中心原为1993年建成的曲阳商务中心，后来在2019年改造成为了商场，比较老气。同时军产地皮的存在限制了板块的商业发展，五角场商圈太强也虹吸了曲阳。\n\n让曲阳的商圈影响力非常低下，基本没啥商业地位。年轻人减少、缺乏优质生源补充后，区域的办学质量也在下降，XQ房没热度，价格涨不起来。\n\n尽管曲阳集中了虹口最强的公办学校，但区内整体教育实力不突出（仅1所“八大”高中，一本率76%），且办学氛围趋于宽松（不鸡血），导致XQ房热度低迷。\n\n加之曲阳的老破小并非纯粹的工人新村，没有历史积累，形成的XQ一直没有杨浦鞍山那么强。在XQ弱的情况下，曲阳XQ房热度也低，老破小XQ溢价只有1万元/m²。\n\n同样是密云小区，放在虹口只能卖4.9万元/m²，放在鞍山能卖6-8万元/m²。杨浦密云小区前几年还卖到过12-14万元/m²，没有对比就没有伤害！\n\n可以说，深度老龄化与人口净流失形成恶性循环，不仅抽空了曲阳板块的购买力和市场活力，还引发了商业、教育双双衰退的连锁反应，加剧了板块下滑态势。\n\n曲阳板块的没落，与其毗邻的大柏树区域产业高地地位的丧失密切相关，同时板块自身也受限于落后的商务环境，几波产业机遇都错过了，加剧了板块地位下滑。\n\n在没有开发北外滩之前，大柏树是虹口的重点发展地区，因而曾经投入了很多产业资源，并有不少外地驻沪机构设置在此，证明了之前其地位很高。\n\n但是后来因为产业升级的需要和虹口发展重心南移，相关的多类重点企业都迁走了，导致区域产业地位下滑明显。\n\n1）首先是大柏树的生产资料与专业市场外迁：大柏树曾是全国性生产资料交易中心，聚集了钢材、建材、家电等专业市场。\n\n比如宝山钢材交易市场（全国前十）、上海商务中心家电城（上海首家专业家电市场），以及好美家建材市场等龙头市场。\n\n但是在后来随着城市产业升级和土地效率的优化，部分低附加值企业迁到了郊区，产业流失了。\n\n大柏树生产资料交易市场交易厅门口\n\n2）再是制造与贸易企业转移：随着市区对环保和产业质量要求的提高，原位于大柏树的机械制造、化工类企业（如上海新力机械制造厂等）逐步外迁。\n\n同时，大宗商品交易平台（如2004年成立的上海大宗钢铁电子交易中心）虽曾落地；但后续因载体空间不足、交通效率低下，未能留住高附加值企业。\n\n3）大柏树的创新产业未能扎根：2000年代末，大柏树虽说布局了前瞻性的数字媒体产业（如国家数字出版基地）；但因办公环境老化、空间割裂（被老破小包围），未能有效吸引头部科技企业，部分入驻企业后续流失。\n\n大柏树传统产业外迁后，高质量企业因交通瓶颈（被内环、中环、中山北一路高架多重阻隔）、载体碎片化（被老破小包围）和虹口发展重心南移（北外滩崛起）未能填补空缺，最终导致产业能级衰退。\n\n大柏树产业地位下滑后，对曲阳形成的购买力在减弱，进而导致整个板块都在下滑。尽管有火车头广场、明珠创意园等更新项目，但产业能级、集聚度和单位产值还是没法与市北高新相比。\n\n缺乏成片、高强度开发的商务规划，仅在老社区中零散“插花”式布局办公楼（如大柏树930科技创意园、中观产业园），难以形成规模效应和吸引力。\n\n进入21世纪后，大柏树地区只有搞了这些零零散散的创意产业园，没形成像徐滨、前滩那样的商务区，错过了机遇。\n\n在曲阳板块内部，形成的是碎片化的低效商务。曲阳商务大厦、海鸥商务大厦等外观和设施明显陈旧，在老化，难再吸引高净值企业入住。\n\n曲阳商务大厦\n\n曲阳零星的办公楼和产业园被大片老破小社区物理割裂，无法形成连片、有规模的商务区，整体商务氛围薄弱。\n\n板块内部路网碎片化（单行道多、红绿灯密、绕行严重），综合交通效率低下，阻碍了商务资源的交互和聚集。\n\n大柏树、曲阳的区域定位已经被边缘化，全区产业重心自然是落在了南部的北外滩，中北部地区长期处于弱势。\n\n邻近的五角场副中心强项在商业而非产业，其辐射力难以有效转化为对曲阳的产业支撑和高端购买力导入。\n\n为了解决板块发展困境，虹口正在从交通、中环发展带寻求突破口，试图二次激活曲阳，未来是不是有扭转颓势的希望呢？\n\n最近曲阳迎来了多个交通大利好，快速路、地铁都来了。上海正在规划一条途经大柏树和曲阳的地下快速南北通道，北起大柏树立交，沿曲阳路-临平路-高阳路-浦东南路-浦三路南下至杨高南路，在曲阳板块北（邯郸路）、南（四平路）设出入口匝道。\n\n这条路对于曲阳的意义非常大：大幅提升板块南北向（尤其是直通陆家嘴）的通行效率，打破交通壁垒；大柏树的枢纽也会相应改造，优化匝道（增南向西、西向南，改建北向南、南向北），增强虹口中部交通枢纽功能，提升区域通达性。\n\n此外，在建的地铁19号线在曲阳南部设上海外国语大学站，将显著拉近曲阳与北外滩、陆家嘴的时空距离，提升区域轨道交通能级和吸引力。\n\n虹口正在打造北中环科创集聚带，将对曲阳进行很大的赋能。在中环崛起战略下，虹口给曲阳带来了一个很大的上海城创金融科技国际产业园项目。\n\n这个产业园定位很高，旨在建设现代化办公楼群，为板块导入金融科技产业，是“提质增能”的关键抓手：上海城创金融科技国际产业园\n\n虹口最近还推出了大柏树商务单元更新提升工程，对包括火车头广场、张江高新区虹口园、明珠创意园等区域进行整体更新提升：更新范围具体实施方案\n\n综合来看，曲阳板块的下滑困境是多重因素长期作用的结果。板块产品端深陷老破小泥潭且更新缓慢；人口端承受着上海最严峻的老龄化与购买力流失；产业端错失机遇，商务环境落后且碎片化。\n\n不过当前规划的交通升级与北中环科创带建设又给板块带来了提升发展的机会。但是板块的老破小实在太多了，次新房缺乏，区域产业能级可能会提高但产品定位改变不了。\n\n曲阳就是一个养老板块，医疗资源丰富、曲阳公园和邻近的鲁迅公园都是休闲好去处。但基础养老配套不是板块的竞争优势，上海像这样的养老板块太多了。\n\n曲阳的未来，恐怕还是要指望北外滩的终极崛起来辐射带动，或者北中环科创带加大能级提升，未来结局究竟如何？让我们拭目以待！"},
    ],
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=2000,
    temperature=0.7,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



system
你是一名地产地图直播领域的专家，长期服务于用户购房决策，擅长根据给定的【背景资料】回答用户提出的问题：
- 从用户的问题和背景资料识别购房者真实意图与隐性焦虑
- 提炼这类购房人群的共性问题
- 根据背景资料用专业但不说教的方式给出能改变认知层级的答案
你的目标不是简单“答题”，而是帮助一类购房用户意识到认知盲区、规避决策风险，并引导其进一步咨询
user
虹口对曲阳板块的更新提升有哪些具体的规划和措施

【背景资料】虹口下滑最严重的地方之一是曲阳。曲阳和大柏树曾是虹口的重点发展区域，曾是产业中心和中产居住地，吸引了众多教师、公务员和产业人群入住，形成了全区最好的居住环境。然而，近年来，虹口全力打造北外滩，导致中北部一些传统居住板块下滑，曲阳便是其中之一。

曲阳的现状不容乐观，面临多方面问题，如二手房产品老化、人口结构失衡、产业流失等。曲阳板块产品不佳，以老破小、老大楼为主，缺乏好房子，是板块竞争力下滑的首要推手。曲阳房龄最高的老破小是50年代建成的玉田新村，其它老破小多在80年代建成。2000年以后建设的“次新”小区非常少，只有北宸佳苑、曲阳豪庭、复城国际、通德苑等少数几个小区。

这些小区规模不大，购买客群多为周边从老公房、老大楼向商品房置换的地缘人群。其中，北宸佳苑、曲阳豪庭是对口虹口四中心小学+虹口实验学校初中的双XQ次新房，地位在板块最高，挂牌单价也处在第一梯队。

曲阳板块的其它产品主要是老破小和老大楼。在众多老破小中，银联花园、东四小区、东五小区、邮电二村、双东小区、建工天峰公寓也是对口虹口四中心小学+虹口实验学校初中的双XQ房。这些小区处在购买价值的第二梯队，成交价在5.4-6万元/m²。

曲阳板块的老破小太多，大部分老公房XQ属性一般、只有自住价值，成交价在4.5-5万元/m²。对于自住的曲阳老破小来说，舒适度比其它片区的老破小更低。但凡是老破小，都具有设施老化、物业管理低下、车位配比少等的通病。

曲阳的老破小劣势更明显，很多小区更小、车位配比更少，市场竞争力下滑更严重。虹口本身地域比较小，开发的小区也普遍偏小。比如松花小区仅295户，小区很小，车位配比还低，只有1：0.07；祥东小区车位配比1：0.06更低，根本没地方停车。

缺点太多后，曲阳的老破小跌幅较大，超过40%。曲阳的老破小在曲阳买个老破小基本上要5万元/m²左右。